In [2]:
import ee
ee.Authenticate()
# Initialize the Earth Engine API
ee.Initialize()

# Define the maskS2clouds function
def maskS2clouds(image):
    qa = image.select('QA60')
    
    # Bits 10 and 11 are clouds and cirrus, respectively.
    cloudBitMask = 1 << 10
    cirrusBitMask = 1 << 11
    
    # Both flags should be set to zero, indicating clear conditions.
    mask = qa.bitwiseAnd(cloudBitMask).eq(0).And(qa.bitwiseAnd(cirrusBitMask).eq(0))
    
    return image.updateMask(mask).divide(10000)

# Load the Sentinel-2 image collection and apply the mask
dataset = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
           .filterDate('2020-01-01', '2020-01-30')
           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
           .map(maskS2clouds))

# Define the visualization parameters
visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

# Set the map center and add the layer to the map
Map = ee.Image(dataset.mean())
Map.setCenter(83.277, 17.7009, 12)
Map.addLayer(Map, visualization, 'RGB')

# Open the map in the Code Editor (if using the GEE Code Editor)
# Map


In [3]:
import requests
from datetime import datetime, timedelta

# Replace 'your_api_key' with your Weatherstack API key
API_KEY = '5d945703771fb95e6a001452f9adb19e'

# Specify the location for which you want the forecast (e.g., 'New York,US')
location = 'New York,US'

# Calculate the date for the next day
tomorrow = datetime.now() + timedelta(days=1)
tomorrow_date = tomorrow.strftime('%Y-%m-%d')

# Define the Weatherstack API endpoint URL
url = f'http://api.weatherstack.com/forecast?access_key={API_KEY}&query={location}&forecast_date={tomorrow_date}'

# Make the API request
response = requests.get(url)

if response.status_code == 200:
    data = response.json()
    
   # Extract the wind speed from the API response
    wind_speed_kmph = data['current']['wind_speed']
    
    print(f"Wind Speed for {tomorrow_date}: {wind_speed_kmph} km/h")
else:
    print(f"Error: Unable to fetch data. Status code {response.status_code}")

Wind Speed for 2023-09-21: 11 km/h
